In [ ]:
shps = {
    #'muncol' : {
        #"shp" : '/mnt/d/rms/results/rms_mun_v10.shp',
        #'uid' : 'uid'
    #},
    'muncol2023' : {
        "shp" : '/mnt/d/rms/results21/rms_mun_v9.shp',
        'uid' : 'uid'
    }
}

refxls = '/mnt/d/rms/lista_indicadores.xlsx'

refsheet = 'ihoje'

yearcol = 'ano'

indicator_id = 'id_indicador'

valuecol = 'valor'

outfile = '/mnt/d/rms/results21/indicadores_mun_12042023.xlsx'

In [ ]:
import os
import pandas as pd
from glass.rd import tbl_to_obj
from glass.rd.shp import shp_to_obj
from glass.wt import obj_to_tbl
from glass.dtr.mge.pd import merge_df

In [ ]:
odfs = []

# Open Referece XLSX
ref_df = tbl_to_obj(refxls, sheet=refsheet)

# Open Shapefiles
for shp in shps:
    shps[shp]['df'] = shp_to_obj(shps[shp]["shp"])

In [ ]:
# Produce out dataframes
for k in shps:
    # For each indicator
    for i, r in ref_df.iterrows():
        # Copy DataFrame
        ndf = shps[k]['df'].copy(deep=True)
        
        # Delete unecessary columns
        mcols = [r[k], shps[k]['uid']]
        dcols = [c for c in ndf.columns.values if c not in mcols]
        ndf.drop(dcols, axis=1, inplace=True)
        
        # Place constants
        ndf['id_indicator'] = r[indicator_id]
        ndf['ano'] = r[yearcol]
        
        # Rename value column and id column
        ndf.rename(columns={
            r[k]           : valuecol,
            shps[k]['uid'] : 'id'
        }, inplace=True)
        
        # Append
        odfs.append(ndf)

In [ ]:
# Merge Dataframes
fdf = merge_df(odfs)

# Write to file
obj_to_tbl(fdf, outfile)